In [1]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from pyphysio import EvenlySignal

Please cite:
Bizzego et al. (2019) 'pyphysio: A physiological signal processing library for data science approaches in physiology', SoftwareX


# primo set di parametri

In [34]:
# primo set di parametri per creare il dizionario

# phasic parameters
a = np.array([8, 14, 18]) # steepness of the onset
b = np.array([10, 15, 20]) # recovery
mu = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]) # time shift
s = np.array([0.100, 0.105, 0.110, 0.115, 0.120, 0.125, 0.130, 0.135, 0.140]) # scale

# creo un vettore con tutte le permutazioni di a,b,mu,s
phasic_vect = np.array(np.meshgrid(a,b,mu,s)).T.reshape(-1,4)

# tonic parameters
Gamma = np.array([-20, -10, 1]) # offset
Delta = np.array([-0.01, -0.009, -0.008, -0.007, -0.006, -0.005, -0.004, -0.003, -0.002, -0.001, 0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1])

# creo un vettore con tutte le permutazioni di Gamma e Delta
tonic_vect = np.array(np.meshgrid(Gamma,Delta)).T.reshape(-1,2)

K1 = len(Gamma)*len(Delta)
K2 = len(a)*len(b)*len(mu)*len(s)
K = K1+K2

print('Gamma:', len(Gamma), 'Delta:', len(Delta), 'tonic atoms:', len(Gamma)*len(Delta))
print('a:', len(a), 'b:', len(b), 'mu:', len(mu), 's:', len(s), 'phasic atoms:', len(a)*len(b)*len(mu)*len(s))
print('tot atoms:', K)

Gamma: 3 Delta: 21 tonic atoms: 63
a: 3 b: 3 mu: 16 s: 9 phasic atoms: 1296
tot atoms: 1359


# secondo set di parametri

In [2]:
# secondo set di parametri

# phasic parameters
a = np.array([0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]) # steepness of the onset
b = np.array([0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2]) # recovery
mu = np.array([0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300,310]) # time shift
s = np.array([0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14]) # scale

K2 = len(a)*len(b)*len(mu)*len(s)

phasic_vect = np.zeros([K2,4])

# creo un vettore con tutte le permutazioni di a,b,mu,s
phasic_vect = np.array(np.meshgrid(a,b,mu,s)).T.reshape(-1,4)

# rimuovo righe in cui a>b oppure per ogni riga in cui a<b popolo un nuovo vettore
pv=np.zeros([K2,4])
for ii in range(0,K2):
    if phasic_vect[ii,0]<phasic_vect[ii,1]:
        pv[ii,:]=phasic_vect[ii,:]

# rimuovo righe con zero
pv = pv[~np.all(pv == 0, axis=1)]
phasic_vect = pv

# tonic parameters
Gamma = np.array([-20,-10,1]) # offset
Delta = np.array([-0.01,-0.009,-0.008,-0.007,-0.006,-0.005,-0.004,-0.003,-0.002,-0.001,0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1])

# creo un vettore con tutte le permutazioni di Gamma e Delta
tonic_vect = np.array(np.meshgrid(Gamma,Delta)).T.reshape(-1,2)

K1 = len(Gamma)*len(Delta)
print('Gamma:', len(Gamma), 'Delta:', len(Delta), 'tonic atoms:', K1)
print('a:', len(a), 'b:', len(b), 'mu:', len(mu), 's:', len(s), 'phasic atoms:', K2, 'phasic atoms with a<b:',len(phasic_vect))
K2 = len(phasic_vect)
K = K1+K2
print('tot atoms:',K)

Gamma: 3 Delta: 21 tonic atoms: 63
a: 19 b: 9 mu: 32 s: 9 phasic atoms: 49248 phasic atoms with a<b: 25920
tot atoms: 25983


In [3]:
# creo un vettore per u
# se t-mu maggiore di 0 u=1

P = 128
t = np.arange(0,P)

u = np.zeros([K2,P])
for row in tqdm(range(0,K2)):
    for col in range(0,P):
        if t[col]-phasic_vect[row,2]>0:
            u[row,col] = 1
print(u.shape)            

100%|██████████| 25920/25920 [00:09<00:00, 2727.19it/s]

(25920, 128)


In [4]:
# creo il dizionario D

D = np.zeros([K,P])

# popolo D con valori di atomi tonic
for row in range(0,K1):
    D[row,:] = tonic_vect[row,0]+tonic_vect[row,1]*t[:]

# popolo D con valori di atomi phasic
for row in range(K1,K):
    D[row,:] = (np.exp(-phasic_vect[row-K1,1]*(phasic_vect[row-K1,3]*t[:]-phasic_vect[row-K1,2]))-np.exp(-phasic_vect[row-K1,0]*(phasic_vect[row-K1,3]*t[:]-phasic_vect[row-K1,2])))*u[row-K1,:]

#D[np.isnan(D)] = 0 # se un valore molto piccolo diventa nan, sostituisco con zero
#D[np.isinf(D)] = 10000000 # se un valore molto grande diventa inf, sostituisco con 1000000
#D[D > 10000000] = 10
#D[D < -10000000] = -10




print(tonic_vect.shape)
print(phasic_vect.shape)
print(D.shape)

(63, 2)
(25920, 4)
(25983, 128)


In [5]:
def scale(X, x_min, x_max):
    nom = (X-X.min(axis=0))*(x_max-x_min)
    denom = X.max(axis=0) - X.min(axis=0)
    denom[denom==0] = 1
    return x_min + nom/denom 

In [7]:
X_scaled = scale(D, -1, 1)
print(X_scaled.shape)

(25983, 128)


In [17]:
print(D)
#np.savetxt("foo.csv", D, delimiter=",")

[[-20.    -20.01  -20.02  ... -21.25  -21.26  -21.27 ]
 [-20.    -20.009 -20.018 ... -21.125 -21.134 -21.143]
 [-20.    -20.008 -20.016 ... -21.    -21.008 -21.016]
 ...
 [  0.      0.      0.    ...   0.      0.      0.   ]
 [  0.      0.      0.    ...   0.      0.      0.   ]
 [  0.      0.      0.    ...   0.      0.      0.   ]]


In [9]:
# prendo il segnale

data = pd.read_csv('/Users/gioelepozzi/Desktop/PMEmo/code/eda_gram/1_EDA.csv', header=None)

subject = data.loc[0,1] # primo soggetto
signal = [] # vettore con i valori eda del primo soggetto
a = data.iloc[:,1]
for k in range(1,len(a)):
    signal.append(a[k])

fs = 50

# trasformazione del segnale per la libreria    
signal_p = EvenlySignal(values = signal, sampling_freq = fs, signal_type = 'eda')

In [10]:
# dividere il segnale in frames

foo = np.array(signal_p) # rendo il segnale un numpy array

# aggiungo tanti zeri alla fine per poter dividere in tot frame uguali da 128 l'uno
foo = np.pad(foo, (0, 54), 'constant')
foo = np.split(foo,13)

# senza aggiungere gli zeri in fondo, lasciando quindi l'ultimo frame più piccolo
#a = np.array_split(foo,13) # dimensione ogni frame 135, tranne ultimo 134 13
#b = np.pad(a)

M = np.array(foo)
print(M.shape)

(13, 128)


In [11]:
# prendo il primo frame

x1 = M[0] 
print('first frame:\n', x1)
# selezione prima colonna M[:,0]

P = len(x1)
print('length of the frame: ', P)

first frame:
 [ 8.08407  8.07339  8.0566   8.04592  8.02303  8.00777  7.99862  7.98488
  7.97573  7.96047  7.94521  7.93911  7.92385  7.91012  7.90249  7.89791
  7.88723  7.8735   7.85976  7.85061  7.83993  7.82467  7.81246  7.80025
  7.78652  7.78957  7.77584  7.77431  7.78499  7.78957  7.80483  7.81246
  7.82925  7.85213  7.87655  7.89944  7.92538  7.96047  7.99862  8.03676
  8.07186  8.11916  8.16646  8.21529  8.26259  8.31753  8.36788  8.41366
  8.46249  8.51131  8.56319  8.60744  8.64864  8.69442  8.73104  8.77834
  8.81801  8.85769  8.876    8.91262  8.93703  8.95687  8.97518  8.99959
  9.0118   9.03927  9.05758  9.07284  9.10335  9.11404  9.1354   9.1476
  9.16134  9.18728  9.19338  9.20559  9.23     9.23458  9.25899  9.26967
  9.29104  9.30324  9.33376  9.35055  9.37191  9.39327  9.41921  9.44057
  9.46804  9.47872  9.50008  9.52602  9.56417  9.59011  9.62673  9.65114
  9.68471  9.72286  9.76101  9.7961   9.83578  9.86782  9.9136   9.95785
 10.0036  10.0509  10.1135  10.1669  1

# OMP

In [18]:
DT = D.transpose()
X_scaledT = X_scaled.transpose()
# Dc = x

# D 128 righe 25983 colonne (trasposto del mio D iniziale)
# c 25983 righe 1 colonna
# x 128 righe 1 colonna

c = np.zeros(K)

print('DT',DT.shape)
print('c',c.shape)
print('x1',x1.shape)

DT (128, 25983)
c (25983,)
x1 (128,)


In [33]:
# OMP implementation 1

from sklearn.linear_model import OrthogonalMatchingPursuit

c = OrthogonalMatchingPursuit(normalize=True,precompute=True).fit(X_scaledT, x1)
#c = OrthogonalMatchingPursuit(normalize=True,precompute=True).fit(DT, x1)
c.score(X_scaledT, x1)
c.predict(X_scaledT)


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/omp.py:673: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  return_n_iter=True)


array([ 8.08355985,  8.0744086 ,  8.05834846,  8.04088571,  8.02459523,
        8.01025285,  7.99759513,  7.9858189 ,  7.9739005 ,  7.96079176,
        7.945533  ,  7.93974544,  7.92165743,  7.91173139,  7.90425972,
        7.89610696,  7.88602292,  7.87410089,  7.86135391,  7.84938668,
        7.84014467,  7.8243421 ,  7.81258348,  7.80059714,  7.79007928,
        7.78225923,  7.77799133,  7.77783093,  7.78209715,  7.79092444,
        7.80430494,  7.81265589,  7.83024642,  7.85069732,  7.87403791,
        7.9002773 ,  7.92940676,  7.96140198,  7.99622501,  8.0338261 ,
        8.07414532,  8.11804736,  8.16701777,  8.21623841,  8.26563327,
        8.31513201,  8.36466956,  8.4141858 ,  8.46362529,  8.51293691,
        8.56207362,  8.60684483,  8.64948816,  8.6927805 ,  8.73558558,
        8.77691557,  8.81591686,  8.85185714,  8.88411353,  8.91216172,
        8.93556609,  8.956159  ,  8.97642654,  8.99665131,  9.01682011,
        9.03692053,  9.05694095,  9.07687048,  9.09669895,  9.11

In [19]:
# OMP implementation 2

from sklearn.linear_model import orthogonal_mp

#c = orthogonal_mp(DT,x1,precompute=True, return_n_iter=True)
c = orthogonal_mp(X_scaledT,x1,precompute=True, return_n_iter=True)


print(c)

[2.07602547 0.         0.         ... 0.         0.         0.        ]


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/omp.py:376: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  return_path=return_path)


In [ ]:
# OMP implementation 3

from sparselandtools.pursuits import OrthogonalMatchingPursuit
from sparselandtools.dictionaries import Dictionary

d = Dictionary(DT)

reg = OrthogonalMatchingPursuit(d, sparsity=1).fit(x1)
z = np.matmul(d.matrix, reg)

In [ ]:
# OMP implementation 4

import omp

npts = 128
domain = np.linspace(0, 1, npts)
y = np.dot(D, x1)

# reconstruct
result = omp.omp(D, y)
print('Solution: %r' % result.coef)

npts = 128
domain = np.linspace(0, 1, npts)
#X = np.array([np.cos(2 * np.pi * m * domain) for m in range(4)]).T
coef = [0., 1., 0., .5]
y = np.dot(D, x1)

plt.figure()
plt.plot(domain, DT)
plt.title('Dictionary atoms')

# reconstruct
result = omp.omp(DT, x1)
print('Solution: %r' % result)